In [1]:
using NBInclude
@nbinclude("datastructs.ipynb")

calculate_distance (generic function with 1 method)

In [2]:
# default dict stores gene ID --> location of gene's head and tail
 
function Base.show(dict::DefaultDict{Int, Vector{Int}}, id_to_char::Dict{Int, Char})
    for (key, value) in dict
        # println("$(key.dna) (id:$(key.id)) => $(value)")
        println(id_to_char[key], " $(key) => $(value)")
    end
end 

In [3]:
# process adj list 
# create dictionary of gene ID --> index/location of gene's head and tail 

function assign_ge_idx_to_gid_to_locdict(ge::GeneEnd, idx::Ref{Int}, gid_to_loc::DefaultDict{Int, Vector{Int}})
    if ge.gene == Telomere() 
        return
    end  

    if ge.head == true
        gid_to_loc[ge.gene.id][2] = idx[]
    else 
        gid_to_loc[ge.gene.id][1] = idx[]
    end
end 

function process_adj_list(adj_list:: Vector{Adjacency})
    geneid_to_location = DefaultDict{Int, Vector{Int}}(() -> zeros(Int, 2)) # tail = idx 1, head = 2 in array
    idx = Ref{Int}(1)

    for adj in adj_list
        assign_ge_idx_to_gid_to_locdict(adj.left, idx, geneid_to_location)
        assign_ge_idx_to_gid_to_locdict(adj.right, idx, geneid_to_location)
        idx[] += 1
    end 
    
    return geneid_to_location 
end 
     

process_adj_list (generic function with 1 method)

In [7]:
# helpers for dcj operations and distance


function other_adjacency_end(ge::GeneEnd, adj::Adjacency)
    if adj.left == ge
        return adj.right
    else
        return adj.left
    end 
end

function other_adjacency_end(t::Telomere, adj::Adjacency)
    return other_adjacency_end(GeneEnd(Telomere()), adj) 
end


# given ge, finds ge in adj list 

# e.g., given adj set: {a:h, b:t} ... {b:h, d:t} & ge = b:t, 
# returns {b:t, d:t} and
function find_next_adj(target_ge::GeneEnd, src_gid_to_l::DefaultDict{Int, Vector{Int}}, src_adjs::Vector{Adjacency})
    (target_ge.head == true) ? th_idx = 2 : th_idx = 1
    
    gene_id = target_ge.gene.id
    src_ge_idx = src_gid_to_l[gene_id][th_idx]
    adj = src_adjs[src_ge_idx]

    if adj.left.gene != Telomere() && adj.left.gene.id == gene_id
        src_ge = adj.left
    elseif adj.right.gene != Telomere() && adj.right.gene.id == gene_id
        src_ge = adj.right 
    end 
    
    return src_ge_idx
end 


function update_adj_set(p::GeneEnd, q::GeneEnd, u_idx::Int, v_idx::Int, src_adjs::Vector{Adjacency}, src_adj_set::Set{Adjacency}) 
    #  replace adj u and v in A by...
    # {p, q} and 
    pq = Adjacency(p, q)

    # u\{p}) U (v\{q}
    other_ge_u = other_adjacency_end(p, src_adjs[u_idx])
    other_ge_v = other_adjacency_end(q, src_adjs[v_idx])
    
    excluding_pq = Adjacency(other_ge_u, other_ge_v) 

    #  remove adj with u, v in src_adj_set 
    delete!(src_adj_set, src_adjs[u_idx])
    delete!(src_adj_set, src_adjs[v_idx])

    #  add adjusted adjacencies to src_adj_set
    push!(src_adj_set, pq)
    push!(src_adj_set, excluding_pq)

    return pq, excluding_pq
end 

function update_adj_set(p::Adjacency, u_idx::Int, src_adjs::Vector{Adjacency}, src_adj_set::Set{Adjacency}) 
    #  replace u in A by {p} and (u\{p})  
    excluding_p_ge = other_adjacency_end(Telomere(), p)
    excluding_p = Adjacency(other_adjacency_end(excluding_p_ge, src_adjs[u_idx]), GeneEnd(Telomere()))
    
    #  remove adj with u, v in src_adj_set 
    delete!(src_adj_set, src_adjs[u_idx])

    #  add p, excluding_p to src_adj_set 
    push!(src_adj_set, p)
    push!(src_adj_set, excluding_p)   

    return p, excluding_p
end 


update_adj_set (generic function with 3 methods)